In [1]:
# %matplotlib ipympl 
import numpy as np
import pandas as pd
import csv
import datetime
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.stats import norm
from scipy.stats import boxcox
from scipy.stats import yeojohnson

In [2]:
# load all data into dataframe
def load_data(path, file_names, aliases):
    dates = {}
    for data_set_idx in range(len(data_files)):
        cur_alias = aliases[data_set_idx]
        with open(path + data_files[data_set_idx] + '.csv', newline='') as csvfile:
            spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
            spamreader.__next__()
            for row in spamreader:
                try:
                    cur_date = datetime.datetime.strptime(row[0], '%m/%d/%Y')

                except Exception as e: 
                   continue
                if not cur_date in dates:
                    dates[cur_date] = {}
#                     # need to generalize here
#                 if data_set_idx == 0 or data_set_idx == 2:
#                     dates[cur_date][cur_alias] = float(row[4])
#                 elif data_set_idx == 1 or data_set_idx == 3 or data_set_idx == 4 or data_set_idx == 5:
                try:
                    dates[cur_date][cur_alias] = float(row[1])
                except:
                    print(row[1])
                    print(cur_alias)
                    print(row)
                    

    df = pd.DataFrame.from_dict(dates, orient='index')
    # df.columns = aliases
    df.reset_index(inplace=True)
    df = df.rename(columns = {'index':'Date'})
    df = df.sort_values('Date')
    df = df.reset_index(drop=True)
    return df


In [3]:
# not_null = df.query(baseline_asset + ".notnull()")
# not_null.reset_index(drop = True, inplace = True)

In [117]:
def load_div_data(asset_list, file_list):
    div_data = {}
    for asset, file_path in zip(asset_list, file_list):
        # Initialize data structure for the asset
        div_data[asset] = {"payment_date": set(), "ex_date": set(), "amount": {}}
        
        # Read CSV file into DataFrame
        if file_path != None:
            df = pd.read_csv(file_path + ".csv", delimiter=',', header=0)
            
            # Iterate over rows in the DataFrame
            for index, row in df.iterrows():
                # Extract relevant data
                ex_date = datetime.datetime.strptime(row["Ex/EFF Date"], '%m/%d/%Y').strftime("%Y-%m-%d")
                cash_amount = row["Cash Amount"]
                payment_date = datetime.datetime.strptime(row["Payment Date"], '%m/%d/%Y').strftime("%Y-%m-%d") 
                
                # Update div_data with extracted data
                div_data[asset]["ex_date"].add(ex_date)
                div_data[asset]["payment_date"].add(payment_date)
                div_data[asset]["amount"][ex_date] = cash_amount
            
    return div_data


def get_x_days_ret(asset, df, div_data, distance, idx):
    start_idx = 0
    end_idx = 0
    if distance < 0:
        distance = abs(distance)
        start_idx = idx - distance
        end_idx = idx + 1
    else:
        start_idx = idx
        end_idx = idx + distance + 1
        
    num_shares = 1
    dollars = 0
    for i in range(start_idx, end_idx):
        if df.iloc[i]["Date"] in div_data["payment_date"]:
            num_shares += dollars / df.iloc[i][asset]
            dollars = 0
        if df.iloc[i]["Date"] in div_data["ex_date"]:
            dollars += div_data["amount"][df.iloc[i]["Date"]] * num_shares
    final_val = num_shares * df.iloc[end_idx - 1][asset] + dollars
    start_val = df.iloc[start_idx][asset]
    return (final_val - start_val) / start_val
            
            
            
    

def add_correlaries_div(cor_assets, cor_days_out, pred_distance, df, assets, div_data):
    # stores percent changes from past x days 
    cors = [[] for i in range(len(cor_assets))]
    # stores percent changes for x future days for each asset
    futs = {}
    for a in assets:
        futs[a] = []
    
    # iterate through all data points
    for idx, row in df.iterrows():
        # past data points
        for alias_idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
            if idx > days_out: # check for enough data
                # get percent change
                time_period_change = get_x_days_ret(asset, df, div_data[asset], -days_out, idx)  
                cors[alias_idx].append(time_period_change)
            else:
                cors[alias_idx].append(None)
        
        #future data
        for asset in assets:
            cur_price = row[asset]
            if idx + pred_distance < df.shape[0] and not pd.isna(cur_price) and not pd.isna(df.iloc[idx + pred_distance][asset]):
                time_period_change = get_x_days_ret(asset, df, div_data[asset], pred_distance, idx)  
                futs[asset].append(time_period_change) 
            else:
                futs[asset].append(None)
    # input into data frame
    for idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
        name = asset + "_" + str(days_out) + "_dys"
        df.insert(df.shape[1], name, cors[idx], True)
    
    for asset in futs.keys():
        name = asset + "_fut_" + str(pred_distance) + "dys"
        df.insert(df.shape[1], name, futs[asset], True)

In [5]:
# adds correlation metrix to dataframe
def add_correlaries(cor_assets, cor_days_out, pred_distance, df, assets):
    # stores percent changes from past x days 
    cors = [[] for i in range(len(cor_assets))]
    # stores percent changes for x future days for each asset
    futs = {}
    for a in assets:
        futs[a] = []
    
    # iterate through all data points
    for idx, row in df.iterrows():
        # past data points
        for alias_idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
            cur_price = row[asset]
            if idx > days_out: # check for enough data
                # get percent change
                last_time_period = df.loc[idx - days_out - 1].at[asset]
                time_period_change = (cur_price - last_time_period)/last_time_period
                cors[alias_idx].append(time_period_change)           
            else:
                cors[alias_idx].append(None)
        
        #future data
        for asset in assets:
            cur_price = row[asset]
            if idx + pred_distance < df.shape[0] and not pd.isna(cur_price) and not pd.isna(df.iloc[idx + pred_distance].at[asset]):
                fut_val = df.iloc[idx + pred_distance].at[asset]
                time_period_change = (fut_val - cur_price)/cur_price  
                futs[asset].append(time_period_change) 
            else:
                futs[asset].append(None)
    # input into data frame
    for idx, (asset, days_out) in enumerate(zip(cor_assets, cor_days_out)):
        name = asset + "_" + str(days_out) + "_dys"
        df.insert(df.shape[1], name, cors[idx], True)
    
    for asset in futs.keys():
        name = asset + "_fut_" + str(pred_distance) + "dys"
        df.insert(df.shape[1], name, futs[asset], True)

In [6]:
def add_pred_differences(pred_distance, baseline_asset, assets, df):
    for idx, asset in enumerate(assets):
#       for idx2, asset2 in enumerate(assets[idx + 1: ]): if you want all differences
        if asset != baseline_asset:
            change_asset = df[asset + "_fut_" + str(pred_distance) + "dys"]
            change_baseline = df[baseline_asset + "_fut_" + str(pred_distance) + "dys"]
            diff = change_asset - change_baseline 
            df.insert(df.shape[1], asset + "_" + baseline_asset + "_" + str(pred_distance) + "dys_diff", diff, True)

In [7]:
# not_null.iloc[5700:5750,[0, 7,8, 9, 10,11,12,13,14,15,16,17,18,19,20]]

In [8]:
# not_null.iloc[-160:,0: 10]

In [9]:
# not_null.columns

In [10]:
# #use sklearn.preprocessing.PowerTransformer instead

# plt.figure()

# column_name = 'sp_20_dys'
# column = not_null[column_name] 
# column = column[~np.isnan(column)]
# print(column)
# # column += np.array([1 for i in range(len(column))])
# # print(column)
# plt.figure()
# plt.hist(column , color = 'red', bins = 500, density=True)
# mean = np.mean(column)
# std = np.std(column)
# print(mean)
# print(std)
# x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))


# plt.figure()
# plt.hist(yeojohnson(column)[0] , color = 'red', bins = 500, density=True)
# mean = np.mean(yeojohnson(column)[0])
# std = np.std(yeojohnson(column)[0])
# print(mean)
# print(std)
# x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))
# # plt.hist(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])) , color = 'red', bins = 500, density=True)
# # mean = np.mean(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])))
# # std = np.std(np.log(sp_not_null[column_name] + np.array([1 for i in range(len(sp_not_null[column_name]))])))



# # column += np.array([1 for i in range(len(column))])
# # column = np.log(column)
# # plt.figure()
# # plt.hist(yeojohnson(column)[0] , color = 'red', bins = 500, density=True)
# # mean = np.mean(yeojohnson(column)[0])
# # std = np.std(yeojohnson(column)[0])
# # print(mean)
# # print(std)
# # x_axis = np.arange(-.3, .3, 0.01)

# plt.plot(x_axis, norm.pdf(x_axis, mean, std))
# # plt.hist(sp_not_null['sp_fut_2wks'], color = 'red', bins = 500)
# # plt.hist(sp_not_null['re_fut_2wks'], color = 'green', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['bnd_fut_2wks'], color = 'blue', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['gld_fut_2wks'], color = 'yellow', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['eu_fut_2wks'], color = 'green', bins = 500, alpha = .5,)
# # plt.hist(sp_not_null['jp_fut_2wks'], color = 'blue', bins = 500, alpha = .5,)
# plt.show()

In [11]:
# fig = plt.figure()
# ax = fig.add_subplot(projection = '3d')

# ax.scatter(not_null["sp_last_month"], not_null["re_last_month"], not_null["re_sp_2wk_diff"])
# ax.set_xlabel('sp_last_month')
# ax.set_ylabel('re_last_month')
# ax.set_zlabel('re_sp_2wk_diff')
# plt.show()

In [12]:
def get_rvs(baseline, df, aliases, pred_distance, print_mats=False):
    rvs = {}
    
    valid_cols = []
    for col in df.columns:
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            valid_cols.append(col)

    valid_cols.append(None)
    
    for asset in aliases:
        if asset != baseline:
            valid_cols[-1] = (asset + "_" + baseline + "_" + str(pred_distance) + "dys_diff")

            cov_mat = df[valid_cols]
            # trying out multiplying every times 100 to get larger covariacnce and then maybe wont get the issue with comuns being 
            # linera
            # cov_mat = cov_mat.multiply(100)
            cov_matrix = pd.DataFrame.cov(cov_mat)
            cov_mat = cov_mat.cov()
            cov_mat = cov_mat.to_numpy()
            if print_mats:
                print(asset)
                print(cov_matrix)


            # means of values
            means = []
            for col in valid_cols:
                means.append(np.mean(df[col]))
            if print_mats:
                print(means)

            rv = multivariate_normal(mean=means, cov=cov_mat, allow_singular=True)
            rvs[asset] = rv
    return rvs
        

import time
def predict(asset, baseline, rv, inputs):

    mu = rv.mean  # Mean vector
    cov = rv.cov
    # Values of the two known variables
    known_values = inputs
    # Indices of the known variables
    known_indices = [i for i in range(len(inputs))]  # indices start from 0
    # Indices of the variable you want to find the expected value for
    unknown_index = [len(inputs)]  # index starts from 0
    # Compute the conditional mean and covariance
    mu_conditional = mu[unknown_index] + cov[unknown_index, known_indices] @ np.linalg.inv(cov[np.ix_(known_indices, known_indices)]) @ (known_values - mu[known_indices])
    # cov_conditional = cov[unknown_index, unknown_index] - np.dot(cov[unknown_index, known_indices], np.linalg.inv(cov[np.ix_(known_indices, known_indices)])) @ cov[np.ix_(known_indices, unknown_index)]
    # Expected value (mean) of the conditional distribution
    return mu_conditional[0]
    


In [13]:
def test_preds(assets, baseline, pred_distance, df_test, df_train):
    rvs = get_rvs(baseline, df_train, assets, pred_distance)
    preds = {}
    actuals = {}
    for asset in assets:
        if asset != baseline:
            preds[asset] = []
            actuals[asset] = []
    
    pred_columns = []
    for col_idx, col in enumerate(df_test.columns):
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            pred_columns.append(col_idx)
            
            
    # print('start')
    for idx, row in df_test.iterrows():
        for asset in assets:
            if asset != baseline:
#                 print(asset)
                col_name = asset + "_" + baseline  + "_" + str(pred_distance) + "dys_diff"
                actual = row[col_name]
                actuals[asset].append(actual)
#                 print(actual)
                columns = []
                pred_input = df_test.iloc[idx, pred_columns]
                if not pred_input.isnull().any():
                    prediction = predict(asset, baseline, rvs[asset], pred_input.tolist())
                    preds[asset].append(prediction)
                    # print(prediction)
                else:
                    preds[asset].append(None)
#                     print(None)
#                 print("---------")
        # if idx % 10== 0:
        #     print(idx)

    return preds, actuals            

In [14]:
# inputs here

# path = 'C:\\Users\\plant\\
path = ''

baseline_asset = 'sp'

file_SP = 'SPY'  
div_SP = 'SPYDividend'
file_RE = 'IYR'
div_RE = 'IYRDividend'
file_BND = 'isharesBondIndexSince2003'
div_BND = 'USAggBondDividend'
file_EU = 'USD_EURHistoricalData'
div_EU = None
file_JP = 'USD_JPYHistoricalData'
div_JP = None
file_GLD = 'GoldFuturesHistoricalData'
div_GLD = None
file_MID = 'IJH'
div_MID = 'IJHDividend'
file_SML = 'IJR'
div_SML = 'IJRDividend'
file_RUT = 'IWM'
div_RUT = 'IWMDividend'
file_EST = 'EZU'
div_EST = 'EZUDividend'
file_EMR = 'EEM'
div_EMR = 'EEMDividend'
file_JST = 'EWJ'
div_JST = 'EWJDividend'

# data_files = [file_name_SP, file_name_RE, file_name_BND, file_name_EU, file_name_JPY, file_name_GOLD, file_name_RUT]
# aliases = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut']

# # input correlaries
# cor_assets = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut', "sp", "re", "bnd", "eu", "jp", "gld", 'rut']
# cor_days_out = [20, 20, 20, 20, 20, 20, 20, 10, 10, 10, 10, 10, 10, 10, 252, 252, 252, 252, 252, 252, 252, 60, 60, 60, 60, 60, 60, 60]
# pred_distance = 10
# assets = ["sp", "re", "bnd", "eu", "jp", "gld", 'rut']


data_files = [file_SP, file_RE, file_BND, file_EU, file_JP, file_GLD, file_MID, file_SML, file_RUT, file_EST, file_EMR, file_JST]
div_files = [div_SP, div_RE, div_BND, div_EU, div_JP, div_GLD, div_MID, div_SML, div_RUT, div_EST, div_EMR, div_JST]
aliases = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]

# input correlaries
# cor_assets = ['sp', "re", "bnd", "eu", "jp", "gld", "sp", "re", "bnd", "eu", "jp", "gld", "sp", "re", "bnd", "eu", "jp", "gld",  "sp", "re", "bnd", "eu", "jp", "gld"]
# cor_days_out = [20,   20,    20,   20,   20,    20,   10,   10,   10, 10,   10,   10,   252,  252, 252, 252,  252,   252,    60,   60,    60,    60,    60,   60]

days_out_list = [1, 5, 10, 20, 60, 120, 252]

cor_assets = []
cor_days_out = []

for a in aliases:
    for d in days_out_list:
        cor_assets.append(a)
        cor_days_out.append(d)

# cor_assets = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]
# cor_days_out = [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
pred_distance = 10
assets = ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]

In [15]:
df = load_data(path, data_files, aliases)


In [16]:
df.iloc[1:2]

,Date,sp,gld,jst,eu,jp,mid,sml,rut,re,est,emr,bnd
1,1990-01-03,NaN,398.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df = df.query(baseline_asset + ".notnull()")
df.reset_index(drop = True, inplace = True)

In [118]:
# add_correlaries(cor_assets, cor_days_out, pred_distance, df, assets)
div_data = load_div_data(aliases, div_files)


In [119]:
print(div_data)

{'sp': {'payment_date': {'2013-04-30', '1998-10-30', '1995-10-31', '2008-01-31', '2007-04-30', '1999-07-30', '2016-10-31', '2000-10-31', '2009-04-30', '2002-07-31', '2017-07-31', '2022-04-29', '1998-07-31', '2023-07-31', '2019-04-30', '2014-01-31', '2020-10-30', '2010-01-29', '2018-07-31', '2013-10-31', '2016-07-29', '2019-10-31', '2007-07-31', '2001-01-31', '2004-01-30', '2015-07-31', '2002-04-30', '2004-10-29', '2005-04-29', '2013-01-31', '2006-10-31', '2019-07-31', '1996-07-31', '2000-07-31', '2017-04-28', '2012-04-30', '2021-10-29', '2010-04-30', '2007-10-31', '2003-04-30', '2020-01-31', '2001-10-31', '2021-01-29', '2022-07-29', '2022-10-31', '2011-10-31', '2000-01-31', '2011-04-29', '2016-04-29', '1999-01-29', '1997-04-30', '2003-07-31', '1996-10-31', '2015-01-30', '1995-07-31', '2012-01-31', '2010-07-30', '2024-01-31', '2021-07-30', '2004-04-30', '2003-10-31', '1997-10-31', '2014-07-31', '2009-10-30', '2020-04-30', '2000-04-28', '2012-07-31', '2006-07-31', '2009-07-31', '2015-04-

In [ ]:
add_correlaries_div(cor_assets, cor_days_out, pred_distance, df, assets, div_data)

In [ ]:
add_pred_differences(pred_distance, baseline_asset, assets, df)

In [ ]:
df.columns

In [ ]:
df.to_csv('AllData.csv', index=False)

In [ ]:
df = df.copy()

In [18]:
df = pd.read_csv('AllData.csv')

In [31]:
df.iloc[3000: 3001, 10 :20]

,est,emr,bnd,sp_1_dys,sp_5_dys,sp_10_dys,sp_20_dys,sp_60_dys,sp_120_dys,sp_252_dys
3000,36.369999,22.064444,102.43,-0.00207,0.009042,0.009972,0.018335,0.078382,0.073962,0.099434


In [53]:
df.iloc[6060: 6066, [20, 25, 100]]

,re_1_dys,re_120_dys,eu_fut_10dys
6060,-0.002507,-0.034110,0.001478
6061,0.004273,-0.031863,0.000529
6062,0.005006,-0.025364,-0.010140
6063,0.005230,-0.028052,-0.006352
6064,-0.003964,-0.038733,-0.002538
6065,-0.003109,-0.047189,-0.017403


In [33]:
columns_to_use = []
dont_use = [ "sp_252_dys" ]
# "sp_252_dys",
aliases_not_use = [ 'sml', 'bnd', 'rut', 'gld', 'eu', 'jp', 'jst', 'est', 'emr', 'mid']
#  ["sp", "re", "bnd", "eu", "jp", "gld", "mid", "sml", "rut", "est", "emr", "jst"]
# rut, emr
time_not_use = [1, 5, 120]
for col in df.columns:
    if col not in dont_use:
        use = True
        for a in aliases_not_use:
            if a in col:
                use = False
        for t in time_not_use:
            if "_" + str(t) + "_" in col:
                use = False
        if use:
            columns_to_use.append(col)
            
aliases_to_use = []
for a in aliases:
    if not a in aliases_not_use:
        aliases_to_use.append(a)

In [34]:
df_test = df[(df['Date'] >= '2018-01-01') & (df['Date'] <= '2018-12-31')][columns_to_use]
df_test = df_test.reset_index(drop=True)
df_train = df[(df['Date'] < '2018-01-01')][columns_to_use]

In [35]:
df_test.shape

(251, 13)

In [36]:
# what I want to change -> currently give percent change of asset in past x number of days
# might want to do the percent difference between spy and that asset in last x days instead
preds, actuals = test_preds(aliases_to_use, baseline_asset, pred_distance, df_test, df_train)

# import pickle 

with open('preds.pkl', 'wb') as f:
    pickle.dump(preds, f)

with open('actuals.pkl', 'wb') as f:
    pickle.dump(actuals, f)



In [37]:
# import pickle
# with open("preds.pkl",'rb') as f:
#     preds = pickle.load(f)
    
# with open("actuals.pkl",'rb') as f:
#     actuals = pickle.load(f)

In [38]:
sign_cor_neg = 0
sign_cor_pos = 0
incor_actual_neg = 0
incor_actual_pos = 0
total_diff = 0
total_count = 0
total_correct = 0

for pred, actual in zip(preds['re'], actuals['re']):
    if pred != None and not pd.isna(actual):
        if pred < 0 and actual < 0:
            sign_cor_neg += 1
            total_correct += 1
        elif pred > 0 and actual > 0:
            sign_cor_pos += 1
            total_correct += 1
        elif actual < 0:
            incor_actual_neg += 1
        else:
            incor_actual_pos +=1
        total_count += 1
        total_diff += abs(pred - actual)

In [39]:
print("pred neg actual neg")
print(sign_cor_neg)
print("pred pos actual pos")
print(sign_cor_pos)
print("pred pos actual neg")
print(incor_actual_neg) 
print("pred neg actual pos")
print(incor_actual_pos )

print("-----")
print("total neg")
print(sign_cor_neg + incor_actual_neg)
print("total pos")
print(sign_cor_pos + incor_actual_pos)
print("----")
print("ave diff")
print(total_diff/total_count)
print("percent correct")
print(total_correct/total_count)


pred neg actual neg
67
pred pos actual pos
68
pred pos actual neg
57
pred neg actual pos
59
-----
total neg
124
total pos
127
----
ave diff
0.022003373743965247
percent correct
0.5378486055776892


In [40]:

all_asset_total = 0
asset_num = 0

for a in aliases_to_use:
    if a != baseline_asset :
    # and a != "eu" and a != "gld" and a != "jp":
        asset_num += 1
        test_asset = a
        print(test_asset)
        
        asset_total = 0
        
        for i in range(pred_distance):
            offset = i
            count = 0
            total = 1
            period_counts = 0
            for date, pred, actual in zip(df_test["Date"][offset:], preds[test_asset][offset:], actuals[test_asset][offset:]):
                if pred != None and not pd.isna(actual):
                    if count == 10:
                        # if pred > .01:
                        if pred > 0:
                            total *= (1 + 10 * pred * actual)
                            # print("long " + str(pred)[0:7] + " " + str(actual)[0:7]+ ": " + str(date) + ": " + str(total))
                        # if pred < -.01:
                        if pred < 0:
                            total *= (1 + 10 * abs(pred) * -(actual))
                            # print("shorted " + str(pred)[0:7] + " " + str(actual)[0:7] +  ": " + str(date) + ": " + str(total))
                        period_counts += 1
                #         else:
                #             total *= (1 + actual)
                        
                        count = 0
                    count += 1
                
                
            # print(offset)
            # print(total)
            asset_total += total
            # print(period_counts)
        print("average")
        print(asset_total / pred_distance)
        all_asset_total += asset_total / pred_distance
print("all asset average")
print(all_asset_total/ (asset_num))

re
average
1.0037851075847128
all asset average
1.0037851075847128


##### 

In [46]:
running_return = 1

for i in range(2006, 2024):
    try:
        df_test = df[(df['Date'] >= str(i) + '-01-01') & (df['Date'] <= str(i) + '-12-31')][columns_to_use]
        df_test = df_test.reset_index(drop=True)
        df_train = df[(df['Date'] < str(i) + '-01-01')][columns_to_use]
        preds, actuals = test_preds(aliases_to_use, baseline_asset, pred_distance, df_test, df_train)
        
        all_asset_total = 0
        asset_num = 0

        print("year: " + str(i))
        for a in aliases_to_use:
            if a != baseline_asset :
                asset_num += 1
                test_asset = a
                # print(test_asset)
                
                asset_total = 0
                
                for i in range(pred_distance):
                    offset = i
                    count = 0
                    total = 1
                    period_counts = 0
                    for date, pred, actual in zip(df_test["Date"][offset:], preds[test_asset][offset:], actuals[test_asset][offset:]):
                        if pred != None and not pd.isna(actual):
                            if count == 10:
                                # if pred > .01:
                                if pred > .01:
                                    total *= (1 + 10 * pred * actual)
                                    # print("long " + str(pred)[0:7] + " " + str(actual)[0:7]+ ": " + str(date) + ": " + str(total))
                                # if pred < -.01:
                                if pred < -0.01:
                                    total *= (1 + 10 * abs(pred) * -(actual))
                                    # print("shorted " + str(pred)[0:7] + " " + str(actual)[0:7] +  ": " + str(date) + ": " + str(total))
                                period_counts += 1
                        #         else:
                        #             total *= (1 + actual)   
                                count = 0
                            count += 1
                    asset_total += total
                if test_asset == 're':
                    print("average")
                    print(asset_total / pred_distance)
                    running_return *= asset_total / pred_distance
                all_asset_total += asset_total / pred_distance
        # print("all asset average")
        # print(all_asset_total/ (asset_num))
    except:
        print(str(i) + " didnt work")
print("running return:")
print(running_return)

year: 2006
average
1.0015773970089312
year: 2007
average
0.978699534834058
year: 2008
average
1.0002990746764548
year: 2009
average
1.0496344394150843
year: 2010
average
1.0022265205357708
year: 2011
average
1.00049803205785
year: 2012
average
1.00557852414927
year: 2013
average
0.9754053684871261
year: 2014
average
1.0027894376311495
year: 2015
average
1.0019889139538898
year: 2016
average
1.000757337955052
year: 2017
average
1.000129072555756
year: 2018
average
1.001369038511884
year: 2019
average
1.0025477769299733
year: 2020
average
1.0032902221169142
year: 2021
average
1.0
year: 2022
average
1.0059322417303025
year: 2023
average
1.0
running return:
1.0314237905611836


Better Testing aparatus

In [69]:
def test_preds_raw(assets, baseline, pred_distance, df_test, df_train):
    rvs = get_rvs(baseline, df_train, assets, pred_distance)
    preds = {}
    actuals = {}
    raw_assets = {}
    raw_baselines = {}
    for asset in assets:
        if asset != baseline:
            raw_assets[asset] = []
            raw_baselines[asset] = []
            preds[asset] = []
            actuals[asset] = []
    
    pred_columns = []
    for col_idx, col in enumerate(df_test.columns):
        if not col in aliases and col != "Date" and not "diff" in col and not "fut" in col:
            pred_columns.append(col_idx)
            
            
    # print('start')
    for idx, row in df_test.iterrows():
        for asset in assets:
            if asset != baseline:
#                 print(asset)
                col_name = asset + "_" + baseline  + "_" + str(pred_distance) + "dys_diff"
                actual = row[col_name]
                actuals[asset].append(actual)

                
                raw_asset = row[asset + "_fut_" + str(pred_distance) + "dys"]
                raw_assets[asset].append(raw_asset)

                raw_baseline = row[baseline + "_fut_" + str(pred_distance) + "dys"]
                raw_baselines[asset].append(raw_baseline)
                
#                 print(actual)
                columns = []
                pred_input = df_test.iloc[idx, pred_columns]
                if not pred_input.isnull().any():
                    prediction = predict(asset, baseline, rvs[asset], pred_input.tolist())
                    preds[asset].append(prediction)
                    # print(prediction)
                else:
                    preds[asset].append(None)
#                     print(None)
#                 print("---------")
        # if idx % 10== 0:
        #     print(idx)

    return preds, actuals, raw_assets, raw_baselines       

In [102]:
running_return = 1
base_return = 1
print(df[(df['Date'] >= '2006-09-12') & (df['Date'] <='2006-09-30')][["Date", 'sp', 're', 'sp_fut_10dys']])

for i in range(2006, 2024):
    # try:
        df_test = df[(df['Date'] >= str(i) + '-01-01') & (df['Date'] <= str(i) + '-12-31')][columns_to_use]
        df_test = df_test.reset_index(drop=True)
        
        df_train = df[(df['Date'] < str(i) + '-01-01')][columns_to_use]
        preds, _, raw_assets, raw_baselines = test_preds_raw(aliases_to_use, baseline_asset, pred_distance, df_test, df_train)
        
        all_asset_total = 0
        asset_num = 0

        print("year: " + str(i))
        for a in aliases_to_use:
            if a != baseline_asset :
                asset_num += 1
                test_asset = a
                # print(test_asset)
                
                asset_total = 0
                base_total = 0
                
                for i in range(pred_distance):
                    offset = i
                    count = 0
                    raw_total = 1
                    raw_base_total = 1
                    period_counts = 0
                    for date, pred, raw_change, base_change in zip(df_test["Date"][offset:], preds[test_asset][offset:], raw_assets[test_asset][offset:], raw_baselines[test_asset][offset:]):
                        if pred != None and not pd.isna(raw_change):
                            if count == 10:
                                # if pred > .01:
                                if pred > 0:
                                    raw_total = raw_total * (10 * pred * (1 + raw_change)) + raw_total * ((1 - 10 * pred) * (1 + base_change))
                                    # print("long " + str(pred)[0:7] + " " + str(actual)[0:7]+ ": " + str(date) + ": " + str(total))
                                # if pred < -.01:
                                if pred < 0:
                                    raw_total = raw_total * (10 * abs(pred) * (base_change - raw_change)) + raw_total * (1 + base_change)
                                    # print("shorted " + str(pred)[0:7] + " " + str(actual)[0:7] +  ": " + str(date) + ": " + str(total))
                                period_counts += 1
                        #         else:
                        #             total *= (1 + actual)   
                                count = 0
                                raw_base_total *= (1 + base_change) 
                            count += 1
                    asset_total += raw_total
                    base_total += raw_base_total
                if test_asset == 're':
                    print("average")
                    print(asset_total / pred_distance)
                    print("base")
                    print(base_total / pred_distance)
                    print(((asset_total / pred_distance) - (base_total / pred_distance)) / (base_total / pred_distance))
                    running_return *= asset_total / pred_distance
                    base_return *= (base_total / pred_distance)
                all_asset_total += asset_total / pred_distance
        # print("all asset average")
        # print(all_asset_total/ (asset_num))
    # except:
    #     print(str(i) + " didnt work")
print("running return:")
print(running_return)
print("base_return")
print(base_return)




            Date          sp         re  sp_fut_10dys
3431  2006-09-12  131.690002  77.300003      0.014352
3432  2006-09-13  132.220001  78.000000      0.011496
3433  2006-09-14  132.229996  78.000000      0.011041
3434  2006-09-15  131.960007  78.199997      0.012276
3435  2006-09-18  132.139999  77.750000      0.007114
3436  2006-09-19  131.809998  78.250000      0.011759
3437  2006-09-20  132.509995  78.000000      0.018187
3438  2006-09-21  131.869995  77.199997      0.025100
3439  2006-09-22  131.470001  77.599998      0.026926
3440  2006-09-25  132.479996  77.550003      0.019701
3441  2006-09-26  133.580002  76.849998      0.012652
3442  2006-09-27  133.740005  77.519997      0.010244
3443  2006-09-28  133.690002  77.250000      0.019373
3444  2006-09-29  133.580002  77.150002      0.022833
year: 2006
average
1.1214195435417518
base
1.1140261687450428
0.006636625785045666
year: 2007
average
0.9553221333976193
base
0.9894973297059556
-0.03453793687193881
year: 2008
average
0.671